In [0]:
USE CATALOG crowd2fund;

CREATE OR REPLACE TEMP VIEW v_business_transactions AS
SELECT t.Date,
       t.`Amount In (GBP)` as Amount_In,
       t.`Amount Out (GBP)` as Amount_Out,
       i.`Business name` as Business_Name
FROM bronze.v_transactions t
INNER JOIN bronze.v_investments i ON t.Date = i.`Investment date` AND t.`Amount Out (GBP)` = i.`Invested (GBP)`
WHERE t.`From/To` LIKE 'Exchange with%'
  AND t.`Amount Out (GBP)` IS NOT NULL
  AND i.`Invested (GBP)` IS NOT NULL
UNION ALL
SELECT t.Date,
       t.`Amount In (GBP)` as Amount_In,
       t.`Amount Out (GBP)` as Amount_Out,
       t.`From/To` as Business_Name
FROM bronze.v_transactions t
WHERE NOT t.`From/To` LIKE 'Exchange with%'
  AND t.`From/To` <> 'Add Funds'
  AND t.`From/To` <> 'Withdrawal Request'


In [0]:
CREATE OR REPLACE FUNCTION default.get_basename(business_name STRING)
RETURNS STRING
RETURN TRIM(
    REGEXP_REPLACE(
      UPPER(TRIM(business_name)), '\s*(LTD|LIMITED|LTD\.|LIMITED\.|PLC|INC)\.?\s*$', '', 1
    )
  );

In [0]:
CREATE OR REPLACE TEMP VIEW v_businesses AS
SELECT
  get_basename(Business_name) AS basename,
  max_by(Business_name, length(Business_name)) AS Business_name
FROM
  v_business_transactions
GROUP BY basename

In [0]:
SELECT * FROM v_businesses

In [0]:
CREATE OR REPLACE TABLE silver.business_transactions AS
SELECT bt.Date, bt.Amount_In, bt.Amount_Out, b.Business_Name
  FROM v_business_transactions bt
  INNER JOIN v_businesses b ON get_basename(bt.Business_name) = b.basename

In [0]:
DESC DETAIL crowd2fund.silver.business_transactions;

In [0]:
OPTIMIZE crowd2fund.silver.business_transactions
ZORDER BY Business_Name;

In [0]:
USE CATALOG crowd2fund;

SELECT
  `Business_name`,
  ROUND(SUM(COALESCE(`Amount_In`, 0)) - SUM(COALESCE(`Amount_Out`, 0)), 2) As Profit
FROM
  silver.business_transactions
WHERE
GROUP BY
  `Business_name`
ORDER BY Profit